In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [7]:
# login information to be passed into HTML requests. DO NOT DIRECTLY PUSH LOGIN INFORMATION TO YOUR GITHUB REPOSITORY!
username = ""
password = ""

payload = {
    "ctl00$cph1$ls1$txtEmail" : username,
    "ctl00$cph1$ls1$txtPassword" : password,
    "ctl00$cph1$ls1$btnLogin": "Login"
}

In [8]:
url = "http://www.eoddata.com/symbols.aspx"

# use Session() to make multiple requests
with requests.Session() as s:
    page = s.get(url)
    soup = BeautifulSoup(page.content)
    # additional form data passed for ASP.NET based website
    payload["__VIEWSTATE"] = soup.select_one("#__VIEWSTATE")["value"]
    payload["__VIEWSTATEGENERATOR"] = soup.select_one("#__VIEWSTATEGENERATOR")["value"]
    payload["__EVENTVALIDATION"] = soup.select_one("#__EVENTVALIDATION")["value"]
    s.post(url, data=payload)
    
    # get .txt formed data
    nyse_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=NYSE")
    nasdaq_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=NASDAQ")
    amex_txt = s.get("http://www.eoddata.com/Data/symbollist.aspx?e=AMEX")

In [9]:
def convert_to_dataframe(text_data, exchange_name):
    row_splitted = text_data.text.split('\r\n')
    exchange = pd.DataFrame([x.split('\t') for x in row_splitted], columns=['Symbol', 'Description'])
    # remove duplicated header and blank row at the end
    exchange = exchange.drop([exchange.index[0], exchange.index[-1]])
    # add which exchange
    name = pd.Series([exchange_name], name='Exchange')
    exchange = exchange.merge(name, how='cross')
    
    return(exchange)

In [10]:
nyse = convert_to_dataframe(nyse_txt, 'NYSE')
nasdaq = convert_to_dataframe(nasdaq_txt, 'NASDAQ')
amex = convert_to_dataframe(amex_txt, 'AMEX')

In [121]:
nyse

,Symbol,Description,exchange
0,A,Agilent Technologies,NYSE
1,AA,Alcoa Corp,NYSE
2,AAC,Ares Acquisition Corp Cl A,NYSE
3,AAC.U,Ares Acquisition Corp Units,NYSE
4,AAC.W,Ares Acquisition Corp WT,NYSE
...,...,...,...
3636,ZTO,Zto Express Cayman Inc ADR,NYSE
3637,ZTR,Virtus Global Divd & Income Fund,NYSE
3638,ZTS,Zoetis Inc Cl A,NYSE
3639,ZUO,Zuora Inc,NYSE


In [122]:
nasdaq

,Symbol,Description,exchange
0,AACG,Ata Creativity Global ADR,NASDAQ
1,AACQ,Artius Acquisition Inc Cl A,NASDAQ
2,AACQU,Artius Acquisition Inc Unit,NASDAQ
3,AACQW,Artius Acquisition Inc WT,NASDAQ
4,AADR,Advisorshares Dorsey Wright ETF,NASDAQ
...,...,...,...
4826,ZWRKU,Z-Work Acquisition Corp Units,NASDAQ
4827,ZWRKW,Z-Work Acquisition Corp WT,NASDAQ
4828,ZY,Zymergen Inc,NASDAQ
4829,ZYNE,Zynerba Pharma CS,NASDAQ


In [123]:
amex

,Symbol,Description,exchange
0,AAA,First Priority Clo Bond ETF,AMEX
1,AAAU,GS Physical Gold ETF,AMEX
2,AADR,Advisorshares Dorsey Wright ETF,AMEX
3,AAMC,Altisource Asset,AMEX
4,AAU,Almaden Minerals,AMEX
...,...,...,...
2450,ZDGE,Zedge Inc Cl B,AMEX
2451,ZIG,The Acquirers Fund ETF,AMEX
2452,ZOM,Zomedica Pharmaceuticals Corp,AMEX
2453,ZROZ,25+ Year Zero Cpn U.S. Trsy Idx ETF Pimco,AMEX


In [11]:
# make as one dataframe
exchange_list = [nasdaq, amex]
stock_list = nyse.append(exchange_list, ignore_index=True)

In [12]:
# check duplicated stocks across three different exchange
stock_list[stock_list.duplicated()]

,Symbol,Description,Exchange


In [13]:
stock_list = stock_list.drop_duplicates(subset=['Symbol'], ignore_index=True)

In [14]:
stock_list.to_csv("stock_list_210701.csv", index=False)

In [15]:
# check change in stock list
old = pd.read_csv("stock_list_210630.csv")

In [19]:
print(f"yesterday : {len(old)}, today : {len(stock_list)}")

yesterday : 10927, today : 10929


In [20]:
changes = pd.merge(old, stock_list, how='outer', indicator=True)
changes[changes['_merge']!='both']

,Symbol,Description,Exchange,_merge
762,CMD,Cantel Medical,NaN,left_only
2409,NTEST.I,NaN,NaN,left_only
3118,STI.U,Northern Star Acquisition [Stic.U],NaN,left_only
3119,STI.W,Northern Star Acquisition [Stic/W],NaN,left_only
3120,STIC,Northern Star Acquisition Cl A,NaN,left_only
4802,DALN,Daleen Technologies Inc.,NaN,left_only
5607,GRUB,Just Eat Takeaway.com N.V. ADR,NaN,left_only
8337,WIFI,Boingo Wireless Inc,NaN,left_only
10927,AQNU,Algonquin Power & Utilities Corp,NYSE,right_only
10928,DDL,Dingdong [Cayman] Limited ADR,NYSE,right_only


In [127]:
# search stock
stock_list[stock_list['Symbol']=='OPTT']

,Symbol,Description,exchange
6889,OPTT,Ocean Power Tech Inc,NASDAQ
